In [1]:
import os, sys 
sys.path.append("/global/home/users/yifengh3/VAE/EMD_VAE")

In [2]:
from utils import *
import os
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

import tensorflow.keras as keras
import tensorflow.keras.backend as K

import os
import os.path as osp
import sys
import json

import numpy as np
#from scipy import linalg as LA

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
# from colorspacious import cspace_converter
from collections import OrderedDict


import tensorflow.keras as keras
import tensorflow.keras.backend as K

import numpy as np
import pandas

2022-09-14 16:44:53.402170: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


1 Physical GPUs, 1 Logical GPUs


2022-09-14 16:44:54.434193: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-09-14 16:44:54.434920: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-09-14 16:44:54.465412: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-09-14 16:44:54.465435: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-09-14 16:44:54.466758: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-09-14 16:44:54.466784: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-0

In [4]:
import utils.VAE_model_tools_param
from utils.VAE_model_tools_param import build_and_compile_annealing_vae, betaVAEModel, reset_metrics, loss_tracker, myTerminateOnNaN

Executor with strength 1 edge matrix:
2022-09-14 16:44:54.958894: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1267]      0 
2022-09-14 16:44:54.958904: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1280] 0:   N 
2022-09-14 16:44:54.962741: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1406] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 22110 MB memory) -> physical GPU (device: 0, name: Quadro RTX 6000, pci bus id: 0000:01:00.0, compute capability: 7.5)


In [5]:
fn =  '/global/home/users/yifengh3/VAE/vec_data/signal/B_signal_vector_muons.h5'
vae_args_file = "/global/home/users/yifengh3/VAE/vec_data/vec_model_no_anneal/vae_args.dat"
model_dir = "/global/home/users/yifengh3/VAE/vec_data/vec_model_no_anneal"
save_name = "reconstructed_B_signal_vector_muons.npz"
numbatches = 1000
no_valid = True

df = pandas.read_hdf(fn,stop=1000000)
print(df.shape)
print("Memory in GB:",sum(df.memory_usage(deep=True)) / (1024**3)+sum(df.memory_usage(deep=True)) / (1024**3))

# Data file contains, for each event, 50 particles (with zero padding), each particle with pT, eta, phi, E.
# we only need first 200th entries, weight is not required here
numparts=50
data = df.values[:,:200]
print(data.shape)
data = data.reshape((-1,numparts,4))
data = data.astype(float)

# Normalize pTs so that HT = 1
HT = np.sum(data[:,:,0],axis=-1)
data[:,:,0] = data[:,:,0]/HT[:,None]
data[:,:,-1] = data[:,:,-1]/HT[:,None]

# Inputs x to NN will be: pT, eta, cos(phi), sin(phi), log E
# Separated phi into cos and sin for continuity around full detector, so make things easier for NN.
# Also adding the log E is mainly because it seems like it should make things easier for NN, since there is an exponential spread in particle energies.
# Feel free to change these choices as desired. E.g. px, py might be equally as good as pt, sin, cos.
sig_input = np.zeros((len(data),numparts,4))
sig_input[:,:,:2] = data[:,:,:2]
sig_input[:,:,2] = np.cos(data[:,:,2])
sig_input[:,:,3] = np.sin(data[:,:,2])
#sig_input[:,:,4] = np.log(data[:,:,3]+1e-8)


data_x = sig_input
# Event 'labels' y are [pT, eta, phi], which is used to calculate EMD to output which is also pT, eta, phi.
data_y = data[:,:,:3]

if no_valid:
    valid_x = data_x
    valid_y = data_y
    valid_ht = HT
else:
    numtrain = 500000
    train_x = data_x[:numtrain]
    train_y = data_y[:numtrain]
    valid_x = data_x[numtrain:numtrain+100000]
    valid_y = data_y[numtrain:numtrain+100000]
    valid_ht = HT[numtrain:numtrain+100000]

# train_output_dir = osp.join(model_dir,"end_beta_checkpoint") #create_dir(osp.join(output_dir, experiment_name))
train_output_dir = model_dir


with open(vae_args_file,'r') as f:
     vae_arg_dict = json.loads(f.read())

print("\n\n vae_arg_dict:", vae_arg_dict)

vae, encoder, decoder = build_and_compile_annealing_vae(**vae_arg_dict)

(163600, 8)
Memory in GB: 0.02194046974182129
(163600, 8)


 vae_arg_dict: {'encoder_conv_layers': [1024, 1024, 1024, 1024], 'dense_size': [1024, 1024, 1024, 1024], 'decoder_sizes': [1024, 1024, 1024, 1024, 1024], 'numIter': 10, 'reg_init': 1.0, 'reg_final': 0.01, 'stopThr': 0.001, 'num_inputs': 4, 'num_particles_in': 50, 'latent_dim': 256, 'verbose': 1, 'dropout': 0.0, 'exponent': 2.0}
Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it with `scale_diag` directly instead.
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 50, 4)]      0                                            
_____________________________________________________________________________________

Model: "VAE"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 50, 4)]      0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 50, 1024)     5120        inputs[0][0]                     
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 50, 1024)     0           conv1d[0][0]                     
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 50, 1024)     1049600     leaky_re_lu[0][0]                
________________________________________________________________________________________________

2022-09-14 16:44:55.484275: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [6]:
def recover_ht(data, ht):
    result = np.array(data, copy=True)
    result[:,:,0] = result[:,:,0]*ht[:,None]
    return result

In [7]:
logbetas = np.linspace(1,-6,50).astype(np.float32)

In [8]:
model_dir = "/global/home/users/yifengh3/VAE/vec_data/vec_model_no_anneal"
model_name = "model_weights_end_1429_betar_1.0e+00.hdf5"

In [9]:
vae.load_weights(os.path.join(model_dir,model_name))

In [10]:
def batch_reconstruct(vae, data, logbeta,batch_size = 200, progress_bar=False):
    num_of_batch = data.shape[0]//batch_size
    result = []
    for i in tqdm(range(num_of_batch+1), disable = not progress_bar):
        x = data[i*batch_size:(i+1)*batch_size]
        result.append(vae.predict_mle([x,np.ones(len(x))*logbeta]))
    return np.vstack(result)

In [11]:
def reconstruct_data(vae, data, beta_range, batch_size = 200):
    result = []
    for logbeta in tqdm(beta_range):
        result.append(batch_reconstruct(
        vae, data, logbeta))
    return np.array(result)

In [12]:
full_result = reconstruct_data(vae, valid_x, logbetas)

100%|███████████████████████████████████████████| 50/50 [00:24<00:00,  2.04it/s]


In [13]:
for i, result in enumerate(full_result):
    full_result[i] = recover_ht(result, valid_ht)
original_data = recover_ht(valid_y, valid_ht)

In [14]:
np.savez(os.path.join(model_dir,save_name), data = original_data, recon = full_result, beta = logbetas)